In [ ]:
# Establish Connection to MySQL Docker Container from Jupyter Notebook

# Step 1: Install MySQL connector
%pip install mysql-connector-python
# Step 2: Import libraries
import mysql.connector
from mysql.connector import Error
 
# Step 3: Connect to MySQL Docker Container
try:
    connection = mysql.connector.connect(
        host='localhost',       # Docker maps to localhost
        port=3307,              # Make sure it's exposed in Docker
        user='root',            # MySQL root user
        password='root',        # Password you set in Docker
        database='Nuclear'       # Your database name
    )
 
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"✅ Connected to MySQL Server version {db_info}")
        cursor = connection.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"🎯 You're connected to database: {record[0]}")
 
except Error as e:
    print(f"❌ Error in connection: {e}")

In [ ]:
#show tables in the database
try:
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print("📋 Tables in database:")
    for table in tables:
        print(table)
 
except Error as e:
    print(f"❌ Query failed: {e}")

In [ ]:
#fetching the structure(schema) of the table
try:
    cursor = connection.cursor()
 
    # Replace 'users' with your table name
    #cursor.execute("SELECT * FROM gamma_split;") #for fetching all data
    cursor.execute("DESCRIBE gamma_split;") #for fetching the structure(schema) of the table
    rows = cursor.fetchall()
 
    print("📋 Data from 'gamma':")
    for row in rows:
        print(row)
 
except Error as e:
    print(f"❌ Query failed: {e}")

In [ ]:
#fetching the data from the MySQL database
try:
    cursor = connection.cursor()
 
    # Replace 'users' with your table name
    cursor.execute("SELECT * FROM gamma_normalised_smoothed;") #for fetching all data
#    cursor.execute("DESCRIBE neutron;") #for fetching the structure(schema) of the table
    rows = cursor.fetchall()
 
    print("📋 Data from 'gamma':")
    i=0
    for row in rows:
        print(row)
        i=i+1
    print(i) #to count the number of rows fetched from the table 
except Error as e:
    print(f"❌ Query failed: {e}")

In [ ]:
#Finding the maximum of each row from the table using pandas and dataframes
#-----------------------------------------------------------------------------
import pandas as pd

# Fetch all data from the table
cursor = connection.cursor()
cursor.execute("SELECT * FROM neutron_split;")
rows = cursor.fetchall()

# Get column names
col_names = [desc[0] for desc in cursor.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=col_names)

# Find maximum of each row
row_max = df.max(axis=1)

# Display or use the result
print(row_max)

In [ ]:
#Finding the maximum of each row from the table using SQL GREATEST() function
# -----------------------------------------------------------------------------

# Get column names from the table
cursor = connection.cursor()
cursor.execute("DESCRIBE neutron_split;")
columns = [row[0] for row in cursor.fetchall()]

# Build the GREATEST() query dynamically
columns_str = ', '.join(columns)
query = f"SELECT GREATEST({columns_str}) AS row_max FROM neutron_split;"

# Execute the query
cursor.execute(query)
row_max_list = [row[0] for row in cursor.fetchall()]

# Display the result
print((row_max_list))

In [ ]:
import pandas as pd
import numpy as np
# Fetch all data from the table
cursor = connection.cursor()
cursor.execute("SELECT * FROM gamma_split;")
rows = cursor.fetchall()

# Get column names
col_names = [desc[0] for desc in cursor.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=col_names)

# Find maximum of each row
row_max = df.max(axis=1)

# Subtract each value from the row max
df_processed = row_max.to_numpy()[:, None] - df.values

# Create a new DataFrame with the same columns
df_processed = pd.DataFrame(df_processed, columns=col_names)
print(df_processed)


#Normalisation of the processed data
#-------------------------------------------------------------
# Min-max normalization for each row
df_np = df_processed.values.astype(float)
row_min = df_np.min(axis=1, keepdims=True)
row_max = df_np.max(axis=1, keepdims=True)
# Avoid division by zero
denom = np.where((row_max - row_min) == 0, 1, row_max - row_min)
df_normalised = (df_np - row_min) / denom

# Create a new DataFrame with the same columns
df_processed_normalised = pd.DataFrame(df_normalised, columns=col_names)
print(df_processed_normalised)
#-----------------------------------------------------------------


#Smoothening the normalised data using moving average
#-------------------------------------------------------------
# Smooth each pulse (row) using moving average with window size 5
window_size = 5

def smooth_row(row, window=window_size):
    return pd.Series(row).rolling(window, min_periods=1, center=True).mean().values

df_smoothed = df_processed.apply(smooth_row, axis=1, result_type='expand')
df_smoothed.columns = df_processed.columns

print(df_smoothed.head())
#-----------------------------------------------------------------

# #without normalisation
# # 1. Create the table in MySQL 
# cursor = connection.cursor()
# col_defs = []
# for col in df_processed.columns:
#     col_defs.append(f"`{col}` INT")  
# col_defs_str = ", ".join(col_defs)
# create_table_sql = f"CREATE TABLE IF NOT EXISTS neutron_processed ({col_defs_str});"
# cursor.execute("DROP TABLE IF EXISTS neutron_processed;")
# cursor.execute(create_table_sql)

# # 2. Prepare data for insertion
# columns = list(df_processed.columns)
# insert_query = f"INSERT INTO neutron_processed ({', '.join(columns)}) VALUES ({', '.join(['%s']*len(columns))});"
# data = df_processed.astype(int).values.tolist()

# # 3. Insert in batches
# batch_size = 50  # Adjust as needed
# try:
#     for start in range(0, len(data), batch_size):
#         end = start + batch_size
#         batch = data[start:end]
#         cursor.executemany(insert_query, batch)
#         connection.commit()
#         print(f"Inserted rows {start} to {end-1}")
#     print("✅ Data successfully inserted into table 'neutron_processed'.")
# except Exception as e:
#     print(f"❌ Failed to insert data into table 'neutron_processed'. Error: {e}")

#without normalisation but smoothened
#-------------------------------------------------------------
#with Smoothening and normalisation
# 1. Create the table in MySQL 
cursor = connection.cursor()
col_defs = []
for col in df_smoothed.columns:
    col_defs.append(f"`{col}` FLOAT")  # Use FLOAT for all columns; adjust if needed
col_defs_str = ", ".join(col_defs)
create_table_sql = f"CREATE TABLE IF NOT EXISTS gamma_smoothed ({col_defs_str});"
cursor.execute("DROP TABLE IF EXISTS gamma_smoothed;")
cursor.execute(create_table_sql)

# 2. Prepare data for insertion
columns = list(df_smoothed.columns)
insert_query = f"INSERT INTO gamma_smoothed ({', '.join(columns)}) VALUES ({', '.join(['%s']*len(columns))});"
data = df_smoothed.astype(float).values.tolist()

# 3. Insert in batches
batch_size = 100  # Adjust as needed
try:
    for start in range(0, len(data), batch_size):
        end = start + batch_size
        batch = data[start:end]
        cursor.executemany(insert_query, batch)
        connection.commit()
        print(f"Inserted rows {start} to {end-1}")
    print("✅ Data successfully inserted into table 'gamma_smoothed'.")
except Exception as e:
    print(f"❌ Failed to insert data into table 'gamma_smoothed'. Error: {e}")
#-----------------------------------------------------------------

# #with normalisation
# # 1. Create the table in MySQL 
# cursor = connection.cursor()
# col_defs = []
# for col in df_processed_normalised.columns:
#     col_defs.append(f"`{col}` FLOAT")  # Use FLOAT for all columns; adjust if needed
# col_defs_str = ", ".join(col_defs)
# create_table_sql = f"CREATE TABLE IF NOT EXISTS gamma_processed_normalised ({col_defs_str});"
# cursor.execute("DROP TABLE IF EXISTS gamma_processed_normalised;")
# cursor.execute(create_table_sql)

# # 2. Prepare data for insertion
# columns = list(df_processed_normalised.columns)
# insert_query = f"INSERT INTO gamma_processed_normalised ({', '.join(columns)}) VALUES ({', '.join(['%s']*len(columns))});"
# data = df_processed_normalised.astype(float).values.tolist()

# # 3. Insert in batches
# batch_size = 100  # Adjust as needed
# try:
#     for start in range(0, len(data), batch_size):
#         end = start + batch_size
#         batch = data[start:end]
#         cursor.executemany(insert_query, batch)
#         connection.commit()
#         print(f"Inserted rows {start} to {end-1}")
#     print("✅ Data successfully inserted into table 'gamma_processed_normalised'.")
# except Exception as e:
#     print(f"❌ Failed to insert data into table 'gamma_processed_normalised'. Error: {e}")

# #with Smoothening and normalisation
# # 1. Create the table in MySQL 
# cursor = connection.cursor()
# col_defs = []
# for col in df_normalised_smoothed.columns:
#     col_defs.append(f"`{col}` FLOAT")  # Use FLOAT for all columns; adjust if needed
# col_defs_str = ", ".join(col_defs)
# create_table_sql = f"CREATE TABLE IF NOT EXISTS neutron_normalised_smoothed ({col_defs_str});"
# cursor.execute("DROP TABLE IF EXISTS neutron_normalised_smoothed;")
# cursor.execute(create_table_sql)

# # 2. Prepare data for insertion
# columns = list(df_normalised_smoothed.columns)
# insert_query = f"INSERT INTO neutron_normalised_smoothed ({', '.join(columns)}) VALUES ({', '.join(['%s']*len(columns))});"
# data = df_normalised_smoothed.astype(float).values.tolist()

# # 3. Insert in batches
# batch_size = 100  # Adjust as needed
# try:
#     for start in range(0, len(data), batch_size):
#         end = start + batch_size
#         batch = data[start:end]
#         cursor.executemany(insert_query, batch)
#         connection.commit()
#         print(f"Inserted rows {start} to {end-1}")
#     print("✅ Data successfully inserted into table 'neutron_normalised_smoothed'.")
# except Exception as e:
#     print(f"❌ Failed to insert data into table 'neutron_normalised_smoothed'. Error: {e}")

In [ ]:
##Plotting the Average plots for neutron and gamma normalised processed data

import pandas as pd
import matplotlib.pyplot as plt

# Extract gamma_processed_normalised
cursor = connection.cursor()
cursor.execute("SELECT * FROM gamma_normalised_smoothed;")
gamma_rows = cursor.fetchall()
gamma_col_names = [desc[0] for desc in cursor.description]
df_gamma = pd.DataFrame(gamma_rows, columns=gamma_col_names)
print(df_gamma.head())
# Extract neutron_processed_normalised
cursor.execute("SELECT * FROM neutron_normalised_smoothed;")
neutron_rows = cursor.fetchall()
neutron_col_names = [desc[0] for desc in cursor.description]
df_neutron = pd.DataFrame(neutron_rows, columns=neutron_col_names)
print(df_neutron.head())
#Calculate mean for each column (index)
gamma_mean = df_gamma.mean(axis=0)
neutron_mean = df_neutron.mean(axis=0)
gamma_mean_norm = (gamma_mean - gamma_mean.min()) / (gamma_mean.max() - gamma_mean.min())
neutron_mean_norm = (neutron_mean - neutron_mean.min()) / (neutron_mean.max() - neutron_mean.min())
# Plot
plt.figure(figsize=(14, 6))
plt.plot(gamma_mean_norm.values[:150], label='Gamma Mean')
plt.plot(neutron_mean_norm.values[:150], label='Neutron Mean')
plt.xlabel('Index')
plt.ylabel('Mean Value')
plt.title('Mean Comparison: Gamma vs Neutron')
plt.legend()
plt.show()

In [ ]:
# Dropping the table if it exists

cursor.execute("SET SESSION wait_timeout = 28800;")
if not connection.is_connected():
    connection.reconnect()
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS gamma_processed_normalised;")
connection.commit()
print("Table dropped.")

In [ ]:
#Dropping specific column from the table

cursor = connection.cursor()
cursor.execute("ALTER TABLE neutron_processed DROP COLUMN t_max;")
connection.commit()
print("Column 't_max' removed from table.")

In [ ]:
#Logistic Regression for classification of neutron and gamma using charge ratios
#-----------------------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 1. Calculate charge ratios for gamma
gamma_short_tail = df_gamma.iloc[:, 0:55].sum(axis=1)   # area under index 30-100
gamma_long_tail = df_gamma.iloc[:, 55:105].sum(axis=1)     # area under index 0-100
gamma_ratio = (gamma_short_tail / gamma_long_tail).values

# 2. Calculate charge ratios for neutron
neutron_short_tail = df_neutron.iloc[:, 0:55].sum(axis=1)
neutron_long_tail = df_neutron.iloc[:, 55:105].sum(axis=1)
neutron_ratio = (neutron_short_tail / neutron_long_tail).values
print(gamma_ratio)
print(neutron_ratio)
# 3. Prepare data and labels
X = np.concatenate([gamma_ratio, neutron_ratio]).reshape(-1, 1)
y = np.concatenate([np.zeros_like(gamma_ratio), np.ones_like(neutron_ratio)])  # 0: gamma, 1: neutron

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. Logistic Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# 6. Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['gamma', 'neutron']))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#Diagnosing the missclassified samples
import matplotlib.pyplot as plt
import numpy as np

# Find indices of misclassified samples
misclassified = np.where(y_test != y_pred)[0]

# Get the predicted and true labels for misclassified samples
misclassified_true = y_test[misclassified]
misclassified_pred = y_pred[misclassified]

# Get the indices in the original dataset (if you used shuffle, this is needed)
# If you used train_test_split with shuffle=True (default), get indices:
_, X_test_idx = train_test_split(
    np.arange(len(X)), test_size=0.2, random_state=42, stratify=y
)
misclassified_orig_idx = X_test_idx[misclassified]

# Separate misclassified gamma and neutron indices
gamma_wrong_idx = misclassified_orig_idx[(misclassified_true == 0)][:5]
neutron_wrong_idx = misclassified_orig_idx[(misclassified_true == 1)][:5]

# Calculate the LR threshold (decision boundary)
# For 1D feature: threshold = -intercept_/coef_
threshold = -clf.intercept_[0] / clf.coef_[0][0]
print(threshold)

# Plot 5 misclassified gamma pulses
plt.figure(figsize=(21, 21))
for i, idx in enumerate(gamma_wrong_idx):
    ratio = X[idx][0]
    plt.subplot(2, 5, i+1)
    plt.plot(df_gamma.iloc[idx % len(df_gamma)].values[:150])
    plt.plot(gamma_mean.values[:150], label='Gamma Mean')
    plt.title(f"Gamma (misclassified)\nIndex: {idx}\nRatio={ratio:.3f}")
    plt.xlabel("Sample Index")
    plt.ylabel("Amplitude")
    plt.legend()
plt.tight_layout()

# Plot 5 misclassified neutron pulses
for i, idx in enumerate(neutron_wrong_idx):
    ratio = X[idx][0]
    plt.subplot(2, 5, 5+i+1)
    plt.plot(df_neutron.iloc[idx % len(df_neutron)].values[:150])
    plt.plot(neutron_mean.values[:150], label='Neutron Mean')
    plt.title(f"Neutron (misclassified)\nIndex: {idx}\nRatio={ratio:.3f}")
    plt.xlabel("Sample Index")
    plt.ylabel("Amplitude")
    plt.legend()
plt.tight_layout()
plt.show()

#plotting the FoM (figure of merit) for the charge ratios
#-----------------------------------------------------------------------------
import matplotlib.pyplot as plt
import numpy as np

# Plot histograms
plt.figure(figsize=(8, 5))
plt.hist(gamma_ratio, bins=500, alpha=0.6, label='Gamma', color='blue', density=True)
plt.hist(neutron_ratio, bins=500, alpha=0.6, label='Neutron', color='orange', density=True)

# Calculate means and FWHMs
mean_gamma = np.mean(gamma_ratio)
mean_neutron = np.mean(neutron_ratio)
std_gamma = np.std(gamma_ratio)
std_neutron = np.std(neutron_ratio)
fwhm_gamma = 2.355 * std_gamma
fwhm_neutron = 2.355 * std_neutron

# Calculate FoM
fom = abs(mean_neutron - mean_gamma) / (fwhm_neutron + fwhm_gamma)

# Annotate means and FoM
plt.axvline(mean_gamma, color='blue', linestyle='--', label=f'Gamma Mean: {mean_gamma:.3f}')
plt.axvline(mean_neutron, color='orange', linestyle='--', label=f'Neutron Mean: {mean_neutron:.3f}')
plt.axvline(threshold, color='red', linestyle=':', label=f'LR Threshold: {threshold:.3f}')
plt.title(f'Pulse Shape Discrimination\nFoM = {fom:.3f}')
plt.xlabel('Charge Ratio')
plt.ylabel('Density')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#To get the best index for short and long tail for classification using logistic regression
#-----------------------------------------------------------------------------
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

best_acc = 0
best_short = None
best_long = None

# Define reasonable search ranges for short and long tail indices
for short_tail in range(30, 80, 5):   # e.g., 10 to 75 in steps of 5
    for long_tail in range(short_tail+10, 120, 5):  # always > short_tail
        # Calculate ratios for gamma
        gamma_short = df_gamma.iloc[:, short_tail:long_tail+1].sum(axis=1)
        gamma_long = df_gamma.iloc[:, 0:long_tail+1].sum(axis=1)
        gamma_ratio = (gamma_short / gamma_long).values

        # Calculate ratios for neutron
        neutron_short = df_neutron.iloc[:, short_tail:long_tail+1].sum(axis=1)
        neutron_long = df_neutron.iloc[:, 0:long_tail+1].sum(axis=1)
        neutron_ratio = (neutron_short / neutron_long).values

        # Prepare data and labels
        X = np.concatenate([gamma_ratio, neutron_ratio]).reshape(-1, 1)
        y = np.concatenate([np.zeros_like(gamma_ratio), np.ones_like(neutron_ratio)])

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=32, stratify=y
        )

        # Logistic Regression
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        # Track best
        if acc > best_acc:
            best_acc = acc
            best_short = short_tail
            best_long = long_tail
            print(f"New best: short={short_tail}, long={long_tail}, accuracy={acc:.4f}")

print(f"\nBest indices: short_tail={best_short}, long_tail={best_long}, accuracy={best_acc:.4f}")

In [ ]:
# code for logistic regression using decay slope comparison
#-----------------------------------------------------------------------------
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

best_acc = 0
best_short = None
best_long = None

# Define reasonable search ranges for short and long tail indices
for short_tail in range(30, 80, 5):   # e.g., 10 to 75 in steps of 5
    for long_tail in range(short_tail+10, 120, 5):  # always > short_tail
        # Calculate ratios for gamma
        gamma_short = df_gamma.iloc[:, short_tail]
        gamma_long = df_gamma.iloc[:, long_tail]

        gamma_ratio = ((gamma_short-gamma_long)/(long_tail-short_tail)).values

        # Calculate ratios for neutron
        neutron_short = df_neutron.iloc[:, short_tail]
        neutron_long = df_neutron.iloc[:, long_tail]

        neutron_ratio = ((neutron_short-neutron_long)/(long_tail-short_tail)).values

        # Prepare data and labels
        X = np.concatenate([gamma_ratio, neutron_ratio]).reshape(-1, 1)
        y = np.concatenate([np.zeros_like(gamma_ratio), np.ones_like(neutron_ratio)])

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=32, stratify=y
        )

        # Logistic Regression
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"short={short_tail}, long={long_tail}, accuracy={acc:.4f}")
        # Track best
        if acc > best_acc:
            best_acc = acc
            best_short = short_tail
            best_long = long_tail
            print(f"New best: short={short_tail}, long={long_tail}, accuracy={acc:.4f}")

print(f"\nBest indices: short_tail={best_short}, long_tail={best_long}, accuracy={best_acc:.4f}")
